모델의 성능을 확인하기 위한 코드이다.

#**1. 구글 드라이브를 코랩에 연결한다.**
> 이는 추후 모델을 불러오고, 학습할 데이터를 불러오기 위해 필요한 과정이다. \
 따라서 이 코드를 실행하기 전에, 구글 드라이브에 모델과 토크나이저, 전처리된 데이터를 업로드 해야 한다.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#**2. 테스트를 위해 필요한 라이브러리를 불러온다.**

In [2]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 43 kB 73 kB/s 
     |████████████████████████████████| 182 kB 63.6 MB/s 
     |████████████████████████████████| 7.6 MB 76.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=dc60519f506f90da25fd9a2896b1565aed94bf37c4eab86ed93ba046e51a92c5
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizerFast, ElectraConfig, ElectraForTokenClassification

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#**3. 테스트 데이터셋을 불러온다.**
> ***데이터의 경로 입력/수정 필수!***

In [6]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/2022_lesik_workspace/lesik/data/total+yejib_test.tsv', sep = '\t', keep_default_na=False)
df.head()

,text,label
0,"1. 어묵과 양파, 청피망은 얇게 채 썰고 대파는 3등분해 길게 채 썰어주세요.",O O CV_INGREDIENT O CV_INGREDIENT O CV_INGREDI...
1,2. 오징어는 깨끗이 씻어 얇게 썬 다음 끓는 물에 살짝 데쳐내주세요.,O O CV_INGREDIENT O O O O O O O O O O O O O O ...
2,"4. 어묵과 오징어, 분량의 양념을 넣고 빠르게 볶아낸 후 대파와 올리고당, 참기름...",O O CV_INGREDIENT O CV_INGREDIENT O O O O O O ...
3,5. 갓 지어낸 솥밥을 접시에 퍼담고 오징어 어묵 볶음을 옆에 예쁘게 담아 완성해주세요.,O O O O O O CV_INGREDIENT O O O O O O O O O O ...
4,2. 만가닥버섯은 밑동 부분을 잘라 먹기 좋게 나누고 양파는 채 썰어주세요.,O O CV_INGREDIENT CV_INGREDIENT CV_INGREDIENT ...


태그는 학습된 모델과 동일해야 되므로 변경해서는 안된다.

In [7]:
# Split labels based on whitespace and turn them into a list
arr_labels = set()
for lb in df.label:
    lb = lb.split()
    for ll in lb:
        if ll not in arr_labels:
            arr_labels.add(ll)

#말뭉치 데이터에 포함된 총 태그
unique_labels = {'OGG_EDUCATION', 'MT_ELEMENT', 'AFW_OTHER_PRODUCTS', 'MT_ROCK', 'TI_OTHERS', 'PS_NAME', 'CV_BUILDING_TYPE', 'AM_REPTILIA', 'OGG_FOOD', 'AF_MUSICAL_INSTRUMENT', 'AF_BUILDING', 'AFA_MUSIC', 'CV_SPORTS_INST', 'QT_ORDER', 'TM_COLOR', 'LCG_MOUNTAIN', 'QT_MAN_COUNT', 'PS_CHARACTER', 'AM_OTHERS', 'OGG_LIBRARY', 'TMM_DISEASE', 'OGG_MEDICINE', 'LCG_ISLAND', 'TI_MINUTE', 'MT_CHEMICAL', 'TM_CELL_TISSUE_ORGAN', 'QT_OTHERS', 'CV_TRIBE', 'QT_TEMPERATURE', 'PT_FLOWER', 'OGG_POLITICS', 'DT_WEEK', 'FD_ART', 'AM_AMPHIBIA', 'FD_MEDICINE', 'AF_CULTURAL_ASSET', 'AF_TRANSPORT', 'EV_SPORTS', 'LCG_CONTINENT', 'PT_TREE', 'TMI_SERVICE', 'AM_MAMMALIA', 'TM_SPORTS', 'CV_INGREDIENT', 'OGG_HOTEL', 'QT_PHONE', 'CV_LANGUAGE', 'CV_FUNDS', 'CV_CURRENCY', 'FD_OTHERS', 'LCG_RIVER', 'LCP_CAPITALCITY', 'LC_OTHERS', 'QT_SIZE', 'TM_CLIMATE', 'TM_SHAPE', 'CV_POLICY', 'EV_ACTIVITY', 'TR_ART', 'QT_ADDRESS', 'OGG_RELIGION', 'CV_POSITION', 'FD_HUMANITIES', 'CV_CULTURE', 'QT_SPORTS', 'QT_ALBUM', 'CV_ART', 'CV_FOOD', 'CV_LAW', 'OGG_MILITARY', 'DT_DAY', 'FD_SOCIAL_SCIENCE', 'LCP_PROVINCE', 'CV_CLOTHING', 'TI_HOUR', 'DT_DYNASTY', 'DT_SEASON', 'FD_SCIENCE', 'TMI_HW', 'OGG_SPORTS', 'TR_OTHERS', 'TM_DIRECTION', 'TMI_SITE', 'QT_LENGTH', 'MT_METAL', 'LCG_OCEAN', 'DT_OTHERS', 'LCP_COUNTY', 'TMIG_GENRE', 'OGG_ECONOMY', 'TMI_SW', 'CV_SPORTS_POSITION', 'AFA_DOCUMENT', 'PT_OTHERS', 'AFA_ART_CRAFT', 'EV_OTHERS', 'TMI_EMAIL', 'QT_PRICE', 'EV_FESTIVAL', 'TI_SECOND', 'CV_TAX', 'O', 'QT_VOLUME', 'AF_WEAPON', 'LCG_BAY', 'OGG_SCIENCE', 'PT_FRUIT', 'CV_OCCUPATION', 'QT_CHANNEL', 'OGG_ART', 'AM_INSECT', 'CV_FOOD_STYLE', 'QT_PERCENTAGE', 'OGG_LAW', 'TR_SCIENCE', 'CV_RELATION', 'AM_PART', 'QT_AGE', 'TMI_MODEL', 'AM_BIRD', 'OGG_OTHERS', 'CV_SPORTS', 'DT_YEAR', 'LCP_COUNTRY', 'AFA_VIDEO', 'DT_GEOAGE', 'TI_DURATION', 'AM_TYPE', 'CV_SEASONING', 'AM_FISH', 'CV_PRIZE', 'PS_PET', 'AFW_SERVICE_PRODUCTS', 'TMI_PROJECT', 'CV_DRINK', 'LC_SPACE', 'LCP_CITY', 'EV_WAR_REVOLUTION', 'AFA_PERFORMANCE', 'QT_SPEED', 'PT_GRASS', 'DT_MONTH', 'PT_PART', 'OGG_MEDIA', 'PT_TYPE', 'TMM_DRUG', 'AF_ROAD', 'DT_DURATION', 'TR_MEDICINE', 'TR_HUMANITIES'}
print(unique_labels)

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

prob_tag_list=[]
for lb in arr_labels:
    if lb not in labels_to_ids:
        prob_tag_list.append("'"+lb+"'")
print(",".join(prob_tag_list))

#말뭉치에 포함되어 있지 않는 태그들 추가
labels_to_ids['CV_ACT'] = 150
ids_to_labels[150] = 'CV_ACT'

labels_to_ids['CV_STATE'] = 151
ids_to_labels[151] = 'CV_STATE'

print(ids_to_labels)
print(len(ids_to_labels))

{'CV_INGREDIENT', 'CV_SEASONING', 'CV_STATE', 'TI_DURATION', 'QT_VOLUME', 'QT_TEMPERATURE', 'O'}
{'TM_DIRECTION', 'OGG_POLITICS', 'TMIG_GENRE', 'DT_GEOAGE', 'LCP_PROVINCE', 'TMI_MODEL', 'AFA_VIDEO', 'CV_CURRENCY', 'OGG_HOTEL', 'EV_OTHERS', 'QT_PERCENTAGE', 'AF_BUILDING', 'FD_MEDICINE', 'CV_FOOD', 'CV_SEASONING', 'OGG_LIBRARY', 'CV_RELATION', 'TR_ART', 'CV_OCCUPATION', 'FD_ART', 'AFW_SERVICE_PRODUCTS', 'CV_SPORTS_INST', 'DT_YEAR', 'DT_DAY', 'TM_COLOR', 'AM_MAMMALIA', 'QT_PHONE', 'CV_CLOTHING', 'TMM_DRUG', 'DT_DYNASTY', 'CV_POSITION', 'TM_SHAPE', 'AM_PART', 'QT_AGE', 'FD_OTHERS', 'TMI_HW', 'MT_ELEMENT', 'OGG_RELIGION', 'OGG_LAW', 'CV_SPORTS', 'LCG_MOUNTAIN', 'TI_SECOND', 'AM_INSECT', 'EV_FESTIVAL', 'QT_SPEED', 'FD_HUMANITIES', 'LCP_COUNTY', 'EV_WAR_REVOLUTION', 'CV_CULTURE', 'CV_PRIZE', 'LC_SPACE', 'AFA_DOCUMENT', 'TI_DURATION', 'TMI_SERVICE', 'QT_MAN_COUNT', 'CV_LANGUAGE', 'QT_TEMPERATURE', 'AF_WEAPON', 'TR_HUMANITIES', 'CV_FUNDS', 'AF_ROAD', 'OGG_SCIENCE', 'CV_INGREDIENT', 'OGG_ECONOMY

In [8]:
# Let's take a look at how can we preprocess the text - Take first example
text = df['text'].values.tolist()
m_len = 0
for t in text:
    if m_len < len(t):
        m_len = len(t)
        
example = text[1]

print(example)
print(m_len)

2. 오징어는 깨끗이 씻어 얇게 썬 다음 끓는 물에 살짝 데쳐내주세요.
172


#**4. 테스트를 진행 할 최종 모델과 토크나이저를 불러온다.**
> ***모델, 토크나이저, epoch 입력/수정 필수!***



In [9]:
def load(epoch):
    model_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/model/FIXED_FINAL_EPOCH_'+ str(epoch)
    model = ElectraForTokenClassification.from_pretrained(model_directory, num_labels=len(labels_to_ids))
    model.to(device)
    
    tokenizer_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/tokenizer/FIXED_FINAL_EPOCH_' +str(epoch)
    tokenizer = ElectraTokenizerFast.from_pretrained(tokenizer_directory)
    
    return model, tokenizer

argument로 불러오기를 원하는 epoch를 적는다.

In [10]:
epoch = 72
model, tokenizer = load(epoch)

#**5. 토큰화를 하기 위해 필요한 코드이다.**
> ***원하는 epoch로 수정 가능!***

In [11]:
from transformers import ElectraTokenizerFast

MAX_LEN = 256
TEST_BATCH_SIZE = 8
EPOCH = 72
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [12]:
class ElectraDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip()
        word_labels = self.data.label[index].split()

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        valid_token_list = []
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] == 0:
                continue
            valid_token_list.append(mapping)
        if len(valid_token_list) != len(word_labels):
            print(index, len(word_labels), len(valid_token_list), sentence)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        if len(labels) != 0:
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] == 0:
                    continue
                tok = tokenizer.convert_ids_to_tokens(encoding['input_ids'][idx])
            
                # overwrite label
                if i == len(labels):
                    break
                encoded_labels[idx] = labels[i]
                i += 1
                
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['label'] = torch.as_tensor(encoded_labels)
        
        return item
    def __len__(self):
        return self.len

In [13]:
testing_set = ElectraDataset(df, tokenizer, MAX_LEN)

In [14]:
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

testing_loader = DataLoader(testing_set, **test_params)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [15]:
model.to(device)

ElectraForTokenClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_fea

In [16]:
inputs = testing_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["label"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.5427, device='cuda:0', grad_fn=<NllLossBackward0>)

In [17]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 256, 152])

In [18]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

#**6. test 함수를 불러오는 섹션이다.**

In [21]:
from sklearn.metrics import classification_report
def test():
    # put model in evaluation mode
    model.eval()
    
    test_loss, test_accuracy = 0, 0
    nb_test_examples, nb_test_steps = 0, 0
    test_preds, test_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['label'].to(device, dtype = torch.long)
            
            output = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = output[0]
            test_logits = output[1]
            
            test_loss += loss.item()

            nb_test_steps += 1
            nb_test_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = test_loss/nb_test_steps
                print(f"Test loss per 100 test steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = test_logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            test_labels.extend(labels)
            test_preds.extend(predictions)
            
            tmp_test_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            test_accuracy += tmp_test_accuracy

    labels = [ids_to_labels[id.item()] for id in test_labels]
    predictions = [ids_to_labels[id.item()] for id in test_preds]
    print(classification_report(labels, predictions))
    test_loss = test_loss / nb_test_steps
    test_accuracy = test_accuracy / nb_test_steps
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

#**TEST 결과**

In [22]:
test()

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


1001 14 13 12. 호박잎은 바닥에 하나씩 펼쳐주세요.
Test loss per 100 test steps: 0.10267500579357147
1145 32 31 10. 달군 프라이팬에 식용유를 두르고 반죽을 한 수저씩 떠 넣으면서 앞뒤로 노릇노릇하게 부쳐줍니다.
840 17 16 14. 식은 약밥은 원하는 크기와 모양으로 자른다.
1112 30 29 11. 콩나물,청경채,버섯,청양고추,후추를 넣어 콩나물이 익을때까지 끓여주어 완성.
823 16 15 10. 냄비에 삶은 고기, 양념을 넣고 버무린다.
1003 99230 23  2229 10. 다진마늘을 넣어서 함께 양념장을 잘 섞이도록 저어주세요.
 16. 묵은지-고기쌈장-양념해놓은밥-돌돌 야무지게 여미듯 말아주세요.
953 18 17 14. 튀긴 고로케는 체반에 올려 기름을 충분히 제거한다.
544 26 25 10. 완성된 단호박 해물 떡 찜을 접시에 담고 먹기 좋게 잘라 맛있게 즐겨주세요.
746 40 39 10. 약한 불에서 팬에 기름을 두른 후 닦아내고 숟가락으로 밀전병 반죽을 한술 떠 팬에 붓고 둥글고 얇게 펼쳐 부쳐주세요.
993 15 14 12. 양념장에 구운두부와 함께 넣어주세요.
815 12 11 10. 끓인 라면은 그릇에 담는다.
496 23 22 11. 접시에 준비한 재료를 둥글게 돌려 담고 가운데 전병을 올려 완성해주세요.
944 26 25 10. 끓어오르면 약불로 줄이고 설탕에 버무린 살를 넣고 5분 정도 끓인다.
Test loss per 100 test steps: 0.10969887605158261
1089 25 24 10. 밥에 참기름1T, 맛소금0.8티스푼을 넣어 잘 섞어줍니다.
856 24 23 10. 익힌 광어는 그릇에 담아 식혀 준 뒤, 마요네즈를 넣고 버무린다.
854 17 16 11. 자른 김 가운데 타원형으로 만든 밥을 넣어 준다.
866 18 17 14. 속을 채운 오징어 끝에 이쑤시개를 꼽아 준다.
994 14 13 13. 구운두부와 양념장을 넣어주세요.


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

 CV_INGREDIENT       0.91      0.96      0.93      3377
  CV_SEASONING       0.89      0.93      0.91      1805
      CV_STATE       0.87      0.94      0.90      1352
  LCP_PROVINCE       0.00      0.00      0.00         0
             O       0.99      0.98      0.98     28804
 QT_PERCENTAGE       0.00      0.00      0.00         0
QT_TEMPERATURE       0.96      0.99      0.97       278
     QT_VOLUME       0.91      0.94      0.93       635
   TI_DURATION       0.97      0.98      0.98       613

      accuracy                           0.97     36864
     macro avg       0.72      0.75      0.73     36864
  weighted avg       0.97      0.97      0.97     36864

Test Loss: 0.11392459023699782
Test Accuracy: 0.9716543158689078


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**(선택) predict 함수이다.**

In [25]:
#예측하고 싶은 텍스트를 넣어주세요.
predict_text = '다진 마늘을 넣고 잘게 자른 김치를 섞어주세요.'

In [23]:
def predict(sentence):
    result = []
    inputs = tokenizer(sentence,
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=MAX_LEN,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    word_prediction = []
    token_prediction = []
    pred_dict = {}
    token_dict = {}
    for token_pred, word_idx in zip(wp_preds, inputs.word_ids()):
        if token_pred[0] not in ['[CLS]','[UNK]','[PAD]','[SEP]']:
            result.append([word_idx, token_pred[0], token_pred[1]]) # 
            #print(token_pred)
            if word_idx not in pred_dict:
                pred_dict[word_idx] = set()
            if word_idx not in token_dict:
                token_dict[word_idx] = ""
            pred_dict[word_idx].add(token_pred[1])
            token_dict[word_idx] += token_pred[0].replace("#", "")
                
    for token_pred, word_idx in zip(wp_preds, inputs.word_ids()):
        #only predictions on first word pieces are important
        if token_pred[0] not in ['[CLS]','[UNK]','[PAD]','[SEP]']:
            if token_pred[1] != 'O':
                token_prediction.append([word_idx, token_pred[0],token_pred[1]])
    
    #for i in range(len(token_prediction)):
    #    print(token_prediction[i], word_idx)
    #print()
    return result

In [26]:
predict(predict_text)

[[0, '다진', 'CV_STATE'],
 [1, '마늘', 'CV_INGREDIENT'],
 [1, '##을', 'O'],
 [2, '넣', 'O'],
 [2, '##고', 'O'],
 [3, '잘', 'CV_STATE'],
 [3, '##게', 'CV_STATE'],
 [4, '자', 'CV_STATE'],
 [4, '##른', 'CV_STATE'],
 [5, '김치', 'CV_INGREDIENT'],
 [5, '##를', 'O'],
 [6, '섞', 'O'],
 [6, '##어', 'O'],
 [6, '##주', 'O'],
 [6, '##세요', 'O'],
 [7, '.', 'O']]